In [ ]:
import torch
import torchtuples as tt
import numpy as np
import pandas as pd
from pycox.models import DeepHitSingle
from pycox.evaluation import EvalSurv
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from pycox.preprocessing.feature_transforms import OrderedCategoricalLong

In [ ]:

# from pycox.models.transforms import MLPVanilla

# Load dataset
file_path = "S1Data.csv"
df = pd.read_csv(file_path)

# Define features and target variables
cols_standardize = ['Age', 'Ejection.Fraction', 'Sodium', 'Creatinine', 'Pletelets', 'CPK']
cols_categorical = ['Gender', 'Smoking', 'Diabetes', 'BP', 'Anaemia']

y_cols = ['TIME', 'Event']
X = df[cols_standardize + cols_categorical]
y = df[y_cols]

# Standardize continuous variables
scaler = StandardScaler()
X[cols_standardize] = scaler.fit_transform(X[cols_standardize])

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

time, event = y_train['TIME'].values, y_train['Event'].values
labtrans = DeepHitSingle.label_transform(10)
y_train = labtrans.fit_transform(time, event)

time_test, event_test = y_test['TIME'].values, y_test['Event'].values
y_test = labtrans.transform(time_test, event_test)

# Define a Transformer-based neural network
def make_model(in_features, num_nodes=[64, 64], out_features=labtrans.out_features):
    return tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm=True, dropout=0.1)

# Initialize DeepHitSingle model
in_features = X_train.shape[1]
net = make_model(in_features)
model = DeepHitSingle(net, tt.optim.Adam(0.01), duration_index=labtrans.cuts)

# Train model
batch_size = 32
epochs = 100
callbacks = [tt.callbacks.EarlyStopping(patience=10)]
loss = model.fit(X_train, y_train, batch_size, epochs, callbacks, val_data=(X_test, y_test))

# Predict survival probabilities
surv = model.predict_surv_df(X_test)

# Evaluate model
ev = EvalSurv(surv, time_test, event_test, censor_surv='km')
c_index = ev.concordance_td()
print("Concordance Index:", c_index)

In [ ]:
# Load the dataset
df = pd.read_csv('S1Data.csv')

In [ ]:
# df.head()
print(df.dtypes)

In [ ]:
# Preprocess data
scaler = StandardScaler()
X = scaler.fit_transform(df.drop(['TIME', 'Event'], axis=1))
durations = df['TIME'].values
events = df['Event'].values

In [ ]:
# Define transformer-based survival model
class SurvivalTransformer(torch.nn.Module):
    def __init__(self, input_dim, num_embeddings=8, num_heads=4):
        super().__init__()
        self.embedding = torch.nn.Linear(input_dim, num_embeddings)
        self.transformer = torch.nn.TransformerEncoderLayer(
            d_model=num_embeddings, nhead=num_heads
        )
        self.output = torch.nn.Linear(num_embeddings, 1)  # Hazard prediction

    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x)
        return self.output(x)

In [ ]:
# Split the data into training and validation sets
X_train, X_val, durations_train, durations_val, events_train, events_val = train_test_split(
    X, durations, events, test_size=0.2, random_state=42
)

# Train the model with explicitly defined validation data
model.fit(
    X_train, 
    (durations_train, events_train), 
    epochs=50, 
    val_data=(X_val, (durations_val, events_val)), 
    batch_size=32
)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, roc_curve
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import warnings
warnings.filterwarnings('ignore')

# 1. Data Loading and Initial Exploration
#----------------------------------------
# Load the CSV data (replace with your actual file loading mechanism if needed)
df = pd.read_csv('S1Data.csv')

# Display basic information about the dataset
print("Dataset Shape:", df.shape)
print("Data Types:\n", df.dtypes)
print("\nMissing Values:\n", df.isnull().sum())
print("\nData Summary:\n", df.describe())

# 2. Data Preprocessing
#---------------------
# Identify target variable, and split features into categorical and numerical
target = 'Event'  # Event is the death event (0=survived, 1=died)
time_column = 'TIME'  # Time is the follow-up period

# Features to use in the model
categorical_features = ['Gender', 'Smoking', 'Diabetes', 'BP', 'Anaemia']
numerical_features = ['Age', 'Ejection.Fraction', 'Sodium', 'Creatinine', 
                      'Pletelets', 'CPK']

# Create X (features) and y (target)
X = df.drop([target, time_column], axis=1)
y = df[target]
time_data = df[time_column]

# Split the data into training and testing sets, stratifying by the target variable
X_train, X_test, y_train, y_test, time_train, time_test = train_test_split(
    X, y, time_data, test_size=0.2, random_state=42, stratify=y
)

# Create preprocessing pipelines for numerical and categorical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Preprocess the data
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# If using a transformer model directly, reshape data to sequence format
# Each patient's data will be treated as a sequence of features
sequence_length = X_train_processed.shape[1]
X_train_seq = X_train_processed.reshape(X_train_processed.shape[0], sequence_length, 1)
X_test_seq = X_test_processed.reshape(X_test_processed.shape[0], sequence_length, 1)

# 3. Exploratory Data Analysis
#----------------------------
def plot_eda():
    # Correlation heatmap for numerical features
    plt.figure(figsize=(12, 10))
    corr = df[numerical_features + [target, time_column]].corr()
    sns.heatmap(corr, annot=True, cmap='coolwarm')
    plt.title('Correlation Heatmap of Numerical Features')
    plt.tight_layout()
    plt.show()
    
    # Distribution of target variable
    plt.figure(figsize=(8, 6))
    sns.countplot(x=target, data=df)
    plt.title('Distribution of Heart Failure Events')
    plt.show()
    
    # Survival analysis visualization
    plt.figure(figsize=(10, 6))
    for event in [0, 1]:
        subset = df[df[target] == event]
        sns.kdeplot(subset[time_column], label=f'Event={event}')
    plt.title('Time Distribution by Event')
    plt.xlabel('Follow-up Time (days)')
    plt.legend(['Survived', 'Died'])
    plt.show()

# Uncomment to run EDA plots
# plot_eda()

# 4. Define Transformer-based Model Architecture
#----------------------------------------------
def build_transformer_model(input_shape, head_size=256, num_heads=4, ff_dim=4, num_transformer_blocks=4, mlp_units=[128, 64], dropout=0.2, mlp_dropout=0.4):
    """
    Build a transformer-based model for the heart failure dataset
    
    Args:
        input_shape: Shape of input data
        head_size: Size of attention heads
        num_heads: Number of attention heads
        ff_dim: Feed forward dimension
        num_transformer_blocks: Number of transformer blocks
        mlp_units: Units in the MLP layer
        dropout: Dropout rate for transformer
        mlp_dropout: Dropout rate for MLP layers
    
    Returns:
        Compiled Keras model
    """
    # Define input
    inputs = keras.Input(shape=input_shape)
    x = inputs
    
    # Create multiple transformer blocks
    for _ in range(num_transformer_blocks):
        # Normalization and Attention
        attention_output = layers.LayerNormalization(epsilon=1e-6)(x)
        attention_output = layers.MultiHeadAttention(
            key_dim=head_size, num_heads=num_heads, dropout=dropout
        )(attention_output, attention_output)
        x = layers.Add()([x, attention_output])
        
        # Feed Forward Network
        ffn_output = layers.LayerNormalization(epsilon=1e-6)(x)
        ffn_output = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(ffn_output)
        ffn_output = layers.Conv1D(filters=input_shape[-1], kernel_size=1)(ffn_output)
        x = layers.Add()([x, ffn_output])
    
    # Global average pooling
    x = layers.GlobalAveragePooling1D()(x)
    
    # MLP for classification
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    
    # Output layer
    outputs = layers.Dense(1, activation="sigmoid")(x)
    
    # Create model
    model = keras.Model(inputs, outputs)
    
    # Compile model with appropriate metrics for survival prediction
    model.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=[
            "accuracy",
            keras.metrics.AUC(),
            keras.metrics.Precision(),
            keras.metrics.Recall()
        ]
    )
    
    return model

# 5. Define alternative Time-To-Event Aware Transformer Model
#----------------------------------------------------------
def build_survival_transformer(input_shape, time_data, head_size=256, num_heads=4, num_transformer_blocks=4):
    """
    Build a survival-specific transformer model that incorporates time-to-event data
    (This is a more specialized approach for survival analysis)
    """
    # Define input
    inputs = keras.Input(shape=input_shape)
    
    # Time embedding (incorporate follow-up time information)
    time_input = keras.Input(shape=(1,))
    time_embedding = layers.Dense(32, activation="relu")(time_input)
    time_embedding = layers.Dense(input_shape[0], activation="relu")(time_embedding)
    time_embedding = layers.Reshape((input_shape[0], 1))(time_embedding)
    
    # Combine feature input with time information
    x = layers.Concatenate(axis=2)([inputs, time_embedding])
    
    # Transformer architecture (similar to previous model)
    # ... (same transformer blocks as in the previous function)
    
    # Specialized survival output
    # This could be modified for different survival analysis approaches
    # (e.g., Cox Proportional Hazards or other survival-specific loss functions)
    
    # For now, similar to the binary classification approach
    outputs = layers.Dense(1, activation="sigmoid")(x)
    
    # Create and compile model
    model = keras.Model([inputs, time_input], outputs)
    model.compile(
        optimizer="adam", 
        loss="binary_crossentropy",
        metrics=["accuracy", keras.metrics.AUC()]
    )
    
    return model

# 6. Model Training and Evaluation
#-------------------------------
def train_and_evaluate_model():
    # Initialize the transformer model
    input_shape = X_train_seq.shape[1:]  # (sequence_length, 1)
    model = build_transformer_model(input_shape)
    
    # Model summary
    model.summary()
    
    # Callbacks for training
    callbacks = [
        keras.callbacks.EarlyStopping(
            patience=10, 
            restore_best_weights=True,
            monitor='val_auc'
        ),
        keras.callbacks.ReduceLROnPlateau(
            factor=0.5, 
            patience=5,
            monitor='val_auc'
        )
    ]
    
    # Train the model
    history = model.fit(
        X_train_seq, 
        y_train,
        validation_split=0.2,
        epochs=100,
        batch_size=32,
        callbacks=callbacks,
        class_weight={0: 1, 1: 2}  # Adjust for class imbalance if needed
    )
    
    # Evaluate the model
    y_pred_proba = model.predict(X_test_seq)
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    # Print evaluation metrics
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    
    print("\nROC-AUC Score:", roc_auc_score(y_test, y_pred_proba))
    
    # Plot ROC Curve
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc_score(y_test, y_pred_proba):.3f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend()
    plt.show()
    
    # Plot training history
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['auc'], label='Training AUC')
    plt.plot(history.history['val_auc'], label='Validation AUC')
    plt.title('AUC')
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    return model, history

# 7. Feature Importance Analysis with Attention Weights
#----------------------------------------------------
def analyze_feature_importance(model, feature_names):
    """
    Extract and visualize feature importance from transformer attention weights
    This is a simplified approach - more sophisticated analyses are possible
    """
    # Get attention weights from the model (this would need to be adapted based on model architecture)
    attention_layer = None
    for layer in model.layers:
        if 'multi_head_attention' in layer.name:
            attention_layer = layer
            break
    
    if attention_layer:
        # Extract attention weights (this is simplified)
        # In a real implementation, you'd run inference and extract attention scores for your data
        weights = attention_layer.get_weights()[0]  # This is approximate
        
        # Create a simplified feature importance visualization
        feature_imp = np.mean(np.abs(weights), axis=1).flatten()
        
        # Map to feature names (the exact mapping would depend on your preprocessing pipeline)
        processed_features = []
        for i, feature in enumerate(numerical_features):
            processed_features.append(feature)
        
        for feature in categorical_features:
            # Approximating the OHE feature names - would need to be adjusted based on actual encoding
            categories = df[feature].unique()
            for cat in categories:
                processed_features.append(f"{feature}_{cat}")
        
        # Trim to match the actual number of features we have importance for
        processed_features = processed_features[:len(feature_imp)]
        
        # Plot feature importance
        plt.figure(figsize=(12, 8))
        importance_df = pd.DataFrame({
            'Feature': processed_features,
            'Importance': feature_imp
        }).sort_values('Importance', ascending=False)
        
        sns.barplot(x='Importance', y='Feature', data=importance_df)
        plt.title('Feature Importance from Transformer Attention')
        plt.tight_layout()
        plt.show()

# 8. Survival Analysis Integration
#-------------------------------
def survival_analysis():
    """
    Additional survival analysis using time-to-event data
    This goes beyond simple classification to analyze survival probabilities over time
    """
    from sklearn.preprocessing import StandardScaler
    from sksurv.ensemble import RandomSurvivalForest
    from sksurv.metrics import concordance_index_censored
    
    # Format data for survival analysis
    structured_array_dtype = np.dtype([
        ('Event', np.bool_),
        ('Time', np.float64)
    ])
    
    y_train_surv = np.zeros(len(y_train), dtype=structured_array_dtype)
    y_train_surv['Event'] = y_train.astype(bool)
    y_train_surv['Time'] = time_train
    
    y_test_surv = np.zeros(len(y_test), dtype=structured_array_dtype)
    y_test_surv['Event'] = y_test.astype(bool)
    y_test_surv['Time'] = time_test
    
    # Use Random Survival Forest as a benchmark survival model
    rsf = RandomSurvivalForest(
        n_estimators=100,
        min_samples_split=10,
        min_samples_leaf=15,
        random_state=42
    )
    
    # Fit on training data
    rsf.fit(X_train_processed, y_train_surv)
    
    # Predict risk scores
    risk_scores = rsf.predict(X_test_processed)
    
    # Calculate Concordance Index
    cindex = concordance_index_censored(
        y_test_surv['Event'],
        y_test_surv['Time'],
        risk_scores
    )
    
    print(f"Concordance Index: {cindex[0]:.4f}")
    
    # You could then compare this with predictions from your transformer model
    # by converting transformer outputs to risk scores

# 9. Main Execution
#----------------
def main():
    """Main execution function to run the workflow"""
    print("1. Data loaded and preprocessed.")
    
    print("\n2. Training Transformer model...")
    model, history = train_and_evaluate_model()
    
    print("\n3. Analyzing feature importance...")
    # This would require additional code to properly extract attention weights
    # analyze_feature_importance(model, processed_features)
    
    print("\n4. Performing survival analysis...")
    # Uncomment to run survival analysis
    # survival_analysis()
    
    print("\n5. Done!")
    return model

# Run the main function to execute the workflow
if __name__ == "__main__":
    model = main()